In [4]:
import pandas as pd

# Generation

In [19]:
df = pd.read_csv("intermediate/generation_by_production_type.csv")
df["Grid Node"] = "UK-GB"
df["unit"] = "MWh"
df = df.rename(columns={
    "Biomass (MW)": "BIOMASS",
    "Hydro Pumped Storage (MW)":"HYDRO_PUMPED_STORAGE",
    "Hydro Run-of-river and poundage (MW)": "HYDRO_RUN_OF_RIVER",
    "Fossil Hard coal (MW)": "HARD_COAL",
    "Fossil Gas (MW)": "OTHER_GAS",
    "Fossil Oil (MW)":"OTHER_OIL",
    "Nuclear (MW)":"NUCLEAR",
    "Other (MW)": "OTHER",
    "Wind Offshore (MW)": "WIND_OFF_SHORE",
    "Wind Onshore (MW)":"WIND_ON_SHORE",
    "Solar (MW)":"SOLAR",

})
df['start_datetime'] = pd.to_datetime(df['datetime']).dt.strftime("%Y-%m-%d %H:%M")
df['end_datetime'] = (pd.to_datetime(df['datetime'])+pd.Timedelta(hours=1)).dt.strftime("%Y-%m-%d %H:%M")
df.drop('datetime',axis=1,inplace=True)
df.to_csv('generation.csv',index=False)

# Exchanges

In [6]:
name_to_grid_node = {
    "IFA": "FR",
    "IFA2": "FR",
    "Moyle": "UK-NIR",
    "BritNEd": "NL",
    "EastWest": "IE",
    "NEMO": "BE",
    "NorthSeaLink": "NO",
}

In [7]:
df = pd.read_csv('intermediate/interconnector_exchanges.csv')
interconnector_names = list(name_to_grid_node.keys())
df = df[interconnector_names + ['datetime']]
# Pivot to datetime and rows for each interconnector
df = df.melt(id_vars='datetime', var_name='interconnector', value_name='Value')

# Map interconnector names to grid nodes and group by datetime
df.loc[:, "Grid Node"] = df.interconnector.map(name_to_grid_node)
df = df.groupby(['datetime', 'Grid Node']).sum(numeric_only=True).reset_index()
df.loc[:, "unit"] = "MW"

df = df.rename(columns={"Grid Node": "Grid Node To"})
df.loc[:, "Grid Node From"] = "UK-GB"

df['start_datetime'] = pd.to_datetime(df['datetime']).dt.strftime("%Y-%m-%d %H:%M")
df['end_datetime'] = (pd.to_datetime(df['datetime'])+pd.Timedelta(hours=1)).dt.strftime("%Y-%m-%d %H:%M")

df.to_csv('imports_exports.csv', index=False)

# Capacity

In [14]:
FUEL_TYPE_MAPPING = {
    "Biomass": "BIOMASS",
    "Fossil Brown coal/Lignite": "BROWN_COAL",
    "Fossil Coal-derived gas": "COAL_DERIVED_GAS",
    "Fossil Gas": "OTHER_GAS",
    "Fossil Hard coal": "HARD_COAL",
    "Fossil Oil shale": "SHALE_OIL",
    "Fossil Oil": "OTHER_OIL",
    "Fossil Peat": "PEAT",
    "Geothermal": "GEOTHERMAL",
    "Hydro Pumped Storage": "HYDRO_PUMPED_STORAGE",
    "Hydro Pumped Storage  - Actual Consumption [MW]": "HYDRO_PUMPED_STORAGE_CONSUMPTION",
    "Hydro Run-of-river and poundage": "HYDRO_RUN_OF_RIVER",
    "Hydro Water Reservoir": "HYDRO_RESERVOIR",
    "Marine": "HYDRO",
    "Nuclear":"NUCLEAR",
    "Other": "OTHER",
    "Other renewable": "OTHER_RENEWABLE",
    "Solar": "SOLAR",
    "Waste": "WASTE",
    "Wind Offshore": "WIND_OFF_SHORE",
    "Wind Onshore": "WIND_ON_SHORE"
}

In [15]:
df = pd.read_csv('intermediate/installed_generation_capacity.csv')
df["Fuel Type"] = df["Fuel Type"].map(FUEL_TYPE_MAPPING)
df = df.rename(columns={"Capacity": "Value"})
df.loc[:, "Grid Node"] = "UK-GB"
df.loc[:, "unit"] = "MW"
df['start_datetime'] = pd.to_datetime(df['datetime']).dt.strftime("%Y-%m-%d %H:%M")
df['end_datetime'] = (pd.to_datetime(df['datetime'])+pd.Timedelta(hours=1)).dt.strftime("%Y-%m-%d %H:%M")
df.to_csv('installed_capacity.csv', index=False)